In [11]:
# Create a minimal, working Gantt demo: CSV input -> PNG output (matplotlib, single plot, no custom colors).

import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 1) Create example CSV with tasks & milestones
csv_path = "example_tasks.csv"
df = pd.DataFrame([
    # id, title, start, end, group, milestone(0/1), depends_on (optional text)
    ["T1", "Cadrage du projet", "2025-09-01", "2025-09-05", "Pilotage", 0, ""],
    ["T2", "Spécifications fonctionnelles", "2025-09-08", "2025-09-19", "Conception", 0, "T1"],
    ["T3", "Architecture technique", "2025-09-15", "2025-09-26", "Conception", 0, "T1"],
    ["M1", "Milestone: Spécifications validées", "2025-09-22", "2025-09-22", "Jalons", 1, "T2,T3"],
    ["T4", "Développement sprint 1", "2025-09-23", "2025-10-10", "Dév", 0, "M1"],
    ["T5", "Développement sprint 2", "2025-10-13", "2025-10-31", "Dév", 0, "T4"],
    ["T6", "Tests & Recette", "2025-11-03", "2025-11-14", "Qualité", 0, "T5"],
    ["M2", "Milestone: Go/No-Go", "2025-11-17", "2025-11-17", "Jalons", 1, "T6"],
])
df.columns = ["id","title","start","end","group","milestone","depends_on"]
df.to_csv(csv_path, index=False)

# 2) Load & validate dates
df["start"] = pd.to_datetime(df["start"])
df["end"] = pd.to_datetime(df["end"])

# 3) Sort by start date
df = df.sort_values("start").reset_index(drop=True)

# 4) Build Gantt plot
fig, ax = plt.subplots(figsize=(12, 6))

# Y positions
y_positions = range(len(df))
ax.set_yticks(list(y_positions))
ax.set_yticklabels(df["title"].tolist())

# Plot tasks (bars) and milestones (markers)
for i, row in df.iterrows():
    start = mdates.date2num(row["start"])
    end = mdates.date2num(row["end"])
    if int(row["milestone"]) == 1 or row["start"].date() == row["end"].date():
        # milestone: draw a vertical marker (triangle) on the date
        ax.plot([start], [i], marker="v")  # no custom colors
    else:
        ax.barh(i, end - start, left=start, height=0.4, align='center')  # default style

# Format x-axis as dates
ax.xaxis_date()
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d %b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

ax.set_xlabel("Dates")
ax.set_ylabel("Activités")
ax.set_title("Planning (exemple)")

plt.tight_layout()

png_path = "planning_exemple.png"
plt.savefig(png_path, dpi=200)
plt.close(fig)

# 5) Also save the script itself for user reuse
script_path = "gantt_from_csv.py"

## Premier test

In [12]:

import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def build_gantt(
    csv_path,
    png_path,
    *,
    # (1) Affichage du titre sur les barres (tâches uniquement)
    show_labels_on_bars=True,
    label_min_days_for_text=1.0,   # seuil pour éviter texte si barre trop courte (en jours)
    label_pad_days=0.0,            # décalage horizontal du label (en jours)
    # (2) Lignes verticales pour milestones
    draw_milestone_vlines=True,
    milestone_vline_linestyle=":",
    milestone_vline_alpha=0.3,
    milestone_marker="v",
    # (3) Bandes horizontales de fond (pour toutes les lignes)
    draw_row_background=True,
    row_bg_alpha=0.1,
    row_bg_height=0.6,             # épaisseur uniforme des bandes (0–1 ~ hauteur en unités Y)
    # Style des barres
    bar_height=0.4,
    # Axe dates
    major_locator_weeks=1,
    date_fmt="%d %b %Y",
    fig_size=(12, 6),
    title="Planning",
):
    """
    Attend un CSV avec colonnes :
    id,title,start,end,group,milestone,depends_on
    - start/end: YYYY-MM-DD
    - milestone: 1 = jalon (start=end), 0 = tâche
    """

    df = pd.read_csv(csv_path)
    # Validation & tri
    df["start"] = pd.to_datetime(df["start"])
    df["end"]   = pd.to_datetime(df["end"])
    df = df.sort_values("start").reset_index(drop=True)

    fig, ax = plt.subplots(figsize=fig_size)

    # Y-positions
    y_positions = range(len(df))
    ax.set_yticks(list(y_positions))
    ax.set_yticklabels(df["title"].tolist())

    # Limites X globales (pour les bandes de fond & vlines)
    x_min = mdates.date2num(df["start"].min())
    x_max = mdates.date2num(df["end"].max())

    # (3) Dessiner les bandes horizontales de fond
    if draw_row_background:
        for i in y_positions:
            y0 = i - row_bg_height / 2.0
            y1 = i + row_bg_height / 2.0
            # Rectangle horizontal couvrant tout l'axe des X (xmin=0, xmax=1 en coordonnées axes)
            ax.axhspan(y0, y1, xmin=0.0, xmax=1.0, alpha=row_bg_alpha, zorder=1)

    # Tracer barres & milestones
    for i, row in df.iterrows():
        start_num = mdates.date2num(row["start"])
        end_num   = mdates.date2num(row["end"])
        is_milestone = int(row.get("milestone", 0)) == 1 or row["start"].date() == row["end"].date()

        if is_milestone:
            # (2) Ligne verticale pointillée transparente
            if draw_milestone_vlines:
                ax.axvline(x=start_num, linestyle=milestone_vline_linestyle,
                           alpha=milestone_vline_alpha, zorder=3)
            # Marqueur pour le jalon
            ax.plot([start_num], [i], marker=milestone_marker, zorder=4)
        else:
            # Barre de tâche
            duration = end_num - start_num
            ax.barh(
                i,
                duration,
                left=start_num,
                height=bar_height,
                align='center',
                zorder=2
            )

            # (1) Titre sur la barre si demandé et si la barre n'est pas trop courte
            if show_labels_on_bars and duration >= label_min_days_for_text:
                label_x = start_num + label_pad_days + duration / 2.0
                ax.text(
                    label_x,
                    i,
                    row["title"],
                    va="center",
                    ha="center",
                    zorder=5,
                    clip_on=True  # évite dépassement à l'export
                )

    # Formatage axe X (dates)
    ax.set_xlim(x_min, x_max)
    ax.xaxis_date()
    if major_locator_weeks is not None:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=major_locator_weeks))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    ax.set_xlabel("Dates")
    ax.set_ylabel("Activités")
    ax.set_title(title)

    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close(fig)


if __name__ == "__main__":
    # Exemple sans arguments : lecture directe d'un fichier par défaut
    input_csv = "example_tasks.csv"
    output_png = "planning_exemple.png"

    build_gantt(
        input_csv,
        output_png,
        show_labels_on_bars=True,
        draw_milestone_vlines=True,
        draw_row_background=True,
        row_bg_alpha=0.1,
        row_bg_height=0.6,
        bar_height=0.4,
        milestone_vline_alpha=0.3,
    )

    print(f"Planning généré : {output_png}")

Planning généré : planning_exemple.png


## Deuxième test

In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def _data_width_in_pixels(ax, x_left, x_right):
    """Convertit une largeur en coordonnées données (dates) en pixels."""
    trans = ax.transData
    p0 = trans.transform((x_left, 0))
    p1 = trans.transform((x_right, 0))
    return abs(p1[0] - p0[0])

def _fit_text_inside_bar(ax, text_str, x_center, y_center, x_left, x_right,
                         max_font_size=12, min_font_size=6, padding_px=6, zorder=5):
    """
    Ajoute un texte centré dans la barre [x_left, x_right] à la coordonnée Y,
    en réduisant la police si nécessaire pour que tout le texte tienne en largeur.
    Retourne True si texte placé, False sinon.
    """
    fig = ax.figure
    # S'assure d'avoir un renderer pour mesurer les tailles
    fig.canvas.draw()
    renderer = fig.canvas.get_renderer()

    available_px = max(_data_width_in_pixels(ax, x_left, x_right) - padding_px, 0)

    # Essaie du plus grand au plus petit
    for fs in range(max_font_size, min_font_size - 1, -1):
        txt = ax.text(
            x_center, y_center, text_str,
            va="center", ha="center",
            fontsize=fs, clip_on=True, zorder=zorder
        )
        bbox = txt.get_window_extent(renderer=renderer)
        text_px = bbox.width
        if text_px <= available_px:
            return True  # taille trouvée
        # sinon, on retire et on réessaie plus petit
        txt.remove()

    # Impossible de faire tenir proprement → on n'affiche pas
    return False

def build_gantt(
    csv_path,
    png_path,
    *,
    # (1) Titre sur les barres seulement (pas d’étiquettes Y)
    show_labels_on_bars=True,
    label_min_days_for_text=0.5,   # ne tente pas le texte si barre trop courte (en jours)
    label_pad_days=0.0,            # décalage horizontal du label (en jours)
    max_font_size=12,
    min_font_size=6,
    label_padding_px=6,
    # (2) Lignes verticales pour milestones
    draw_milestone_vlines=True,
    milestone_vline_linestyle=":",
    milestone_vline_alpha=0.3,
    milestone_marker="v",
    # (3) Bandes horizontales de fond
    draw_row_background=True,
    row_bg_alpha=0.1,
    row_bg_height=0.6,             # épaisseur (0–1 environ) autour de la ligne Y
    # Style des barres
    bar_height=0.4,
    # Axe dates
    major_locator_weeks=1,
    date_fmt="%d %b %Y",
    fig_size=(12, 6),
    title="Planning",
):
    """
    CSV requis : id,title,start,end,group,milestone,depends_on
    - start/end: YYYY-MM-DD
    - milestone: 1 = jalon (start=end), 0 = tâche
    """

    df = pd.read_csv(csv_path)
    df["start"] = pd.to_datetime(df["start"])
    df["end"]   = pd.to_datetime(df["end"])
    df = df.sort_values("start").reset_index(drop=True)

    fig, ax = plt.subplots(figsize=fig_size)

    # Y ticks sans libellés (on n’affiche pas les titres en Y)
    y_positions = range(len(df))
    ax.set_yticks(list(y_positions))
    ax.set_yticklabels([])  # <<<<<<<<<< pas de titres en ordonnée

    # Limites X globales
    x_min = mdates.date2num(df["start"].min())
    x_max = mdates.date2num(df["end"].max())

    # (3) Bandes de fond pleine largeur
    if draw_row_background:
        for i in y_positions:
            y0 = i - row_bg_height / 2.0
            y1 = i + row_bg_height / 2.0
            ax.axhspan(y0, y1, xmin=0.0, xmax=1.0, alpha=row_bg_alpha, zorder=1)

    # Tracé des barres et jalons
    for i, row in df.iterrows():
        start_num = mdates.date2num(row["start"])
        end_num   = mdates.date2num(row["end"])
        is_milestone = int(row.get("milestone", 0)) == 1 or row["start"].date() == row["end"].date()

        if is_milestone:
            # (2) Ligne verticale + marqueur
            if draw_milestone_vlines:
                ax.axvline(x=start_num, linestyle=milestone_vline_linestyle,
                           alpha=milestone_vline_alpha, zorder=3)
            ax.plot([start_num], [i], marker=milestone_marker, zorder=4)
        else:
            # Barre
            duration = end_num - start_num
            ax.barh(
                i,
                duration,
                left=start_num,
                height=bar_height,
                align='center',
                zorder=2
            )

            # Titre sur la barre uniquement
            if show_labels_on_bars and duration >= label_min_days_for_text:
                label_x = start_num + label_pad_days + duration / 2.0
                # Ajuste dynamiquement la taille pour que tout le texte tienne
                _fit_text_inside_bar(
                    ax,
                    row["title"],
                    label_x,
                    i,
                    start_num + label_pad_days,
                    end_num,
                    max_font_size=max_font_size,
                    min_font_size=min_font_size,
                    padding_px=label_padding_px,
                    zorder=5
                )

    # Axe X (dates)
    ax.set_xlim(x_min, x_max)
    ax.xaxis_date()
    if major_locator_weeks is not None:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=major_locator_weeks))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    ax.set_xlabel("Dates")
    ax.set_ylabel("")  # plus d'intitulé "Activités" nécessaire si titres sur barres
    ax.set_title(title)

    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close(fig)

if __name__ == "__main__":
    # Exemple sans arguments : lecture directe d'un fichier par défaut
    input_csv = "example_tasks.csv"
    output_png = "planning_exemple.png"

    build_gantt(
        input_csv,
        output_png,
        show_labels_on_bars=True,
        draw_milestone_vlines=True,
        draw_row_background=True,
        row_bg_alpha=0.1,
        row_bg_height=0.6,
        bar_height=0.4,
        milestone_vline_alpha=0.3,
        # Texte dynamique
        max_font_size=12,
        min_font_size=6,
        label_padding_px=6,
        label_min_days_for_text=0.5,
    )

    print(f"Planning généré : {output_png}")

Planning généré : planning_exemple.png


## Troisime test

In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def _data_width_in_pixels(ax, x_left, x_right):
    """Convertit une largeur en coordonnées données (dates) en pixels."""
    trans = ax.transData
    p0 = trans.transform((x_left, 0))
    p1 = trans.transform((x_right, 0))
    return abs(p1[0] - p0[0])

def _fit_text_inside_bar(ax, text_str, x_center, y_center, x_left, x_right,
                         max_font_size=12, min_font_size=6, padding_px=6, zorder=5):
    """
    Ajoute un texte centré dans la barre [x_left, x_right] à la coordonnée Y,
    en réduisant la police si nécessaire pour que tout le texte tienne en largeur.
    Retourne True si texte placé, False sinon.
    """
    fig = ax.figure
    fig.canvas.draw()  # assure un renderer pour mesurer
    renderer = fig.canvas.get_renderer()

    available_px = max(_data_width_in_pixels(ax, x_left, x_right) - padding_px, 0)

    for fs in range(max_font_size, min_font_size - 1, -1):
        txt = ax.text(
            x_center, y_center, text_str,
            va="center", ha="center",
            fontsize=fs, clip_on=True, zorder=zorder
        )
        bbox = txt.get_window_extent(renderer=renderer)
        if bbox.width <= available_px:
            return True
        txt.remove()

    return False

def build_gantt(
    csv_path,
    png_path,
    *,
    # (1) Titre sur les barres seulement (pas d’étiquettes Y)
    show_labels_on_bars=True,
    label_min_days_for_text=0.5,   # ne tente pas le texte si barre trop courte (en jours)
    label_pad_days=0.0,            # décalage horizontal du label (en jours)
    max_font_size=12,
    min_font_size=6,
    label_padding_px=6,
    # (2) Lignes verticales pour milestones
    draw_milestone_vlines=True,
    milestone_vline_linestyle=":",
    milestone_vline_alpha=0.3,
    milestone_marker="v",
    # (3) Bandes horizontales de fond
    draw_row_background=True,
    row_bg_alpha=0.1,
    row_bg_height=0.6,             # épaisseur (0–1 environ) autour de la ligne Y
    # Style des barres
    bar_height=0.4,
    # Axe dates
    major_locator_weeks=1,
    date_fmt="%d %b %Y",
    fig_size=(12, 6),
    title="Planning",
):
    """
    CSV requis : id,title,start,end,group,milestone,depends_on
    - start/end: YYYY-MM-DD
    - milestone: 1 = jalon (start=end), 0 = tâche
    """

    df = pd.read_csv(csv_path)
    df["start"] = pd.to_datetime(df["start"])
    df["end"]   = pd.to_datetime(df["end"])
    df = df.sort_values("start").reset_index(drop=True)

    fig, ax = plt.subplots(figsize=fig_size)

    # Y sans labels ni ticks (pas d’axe vertical affiché)
    y_positions = range(len(df))
    ax.set_yticks(list(y_positions))
    ax.set_yticklabels([])
    ax.tick_params(axis='y', left=False, labelleft=False)

    # Limites X globales
    x_min = mdates.date2num(df["start"].min())
    x_max = mdates.date2num(df["end"].max())

    # (3) Bandes de fond pleine largeur
    if draw_row_background:
        for i in y_positions:
            y0 = i - row_bg_height / 2.0
            y1 = i + row_bg_height / 2.0
            ax.axhspan(y0, y1, xmin=0.0, xmax=1.0, alpha=row_bg_alpha, zorder=1)

    # Tracé des barres et jalons
    for i, row in df.iterrows():
        start_num = mdates.date2num(row["start"])
        end_num   = mdates.date2num(row["end"])
        is_milestone = int(row.get("milestone", 0)) == 1 or row["start"].date() == row["end"].date()

        if is_milestone:
            # (2) Ligne verticale + marqueur
            if draw_milestone_vlines:
                ax.axvline(x=start_num, linestyle=milestone_vline_linestyle,
                           alpha=milestone_vline_alpha, zorder=3)
            ax.plot([start_num], [i], marker=milestone_marker, zorder=4)
        else:
            # Barre
            duration = end_num - start_num
            ax.barh(
                i,
                duration,
                left=start_num,
                height=bar_height,
                align='center',
                zorder=2
            )

            # Titre sur la barre uniquement, police auto-ajustée
            if show_labels_on_bars and duration >= label_min_days_for_text:
                label_x = start_num + label_pad_days + duration / 2.0
                _fit_text_inside_bar(
                    ax,
                    row["title"],
                    label_x,
                    i,
                    start_num + label_pad_days,
                    end_num,
                    max_font_size=max_font_size,
                    min_font_size=min_font_size,
                    padding_px=label_padding_px,
                    zorder=5
                )

    # Axe X (dates)
    ax.set_xlim(x_min, x_max)
    ax.xaxis_date()
    if major_locator_weeks is not None:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=major_locator_weeks))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    # (1) Supprimer l’encadrement (toutes les spines)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # (2) Pas d’axe vertical explicite déjà géré plus haut (ticks/labels Y off)

    # (3) Remplacer la “ligne d’axe X” par une flèche rouge épaisse
    #     On dessine une flèche au bas du graphique, de x_min à x_max.
    ax.annotate(
        '', 
        xy=(x_max, 0.0), xycoords=('data', 'axes fraction'),
        xytext=(x_min, 0.0), textcoords=('data', 'axes fraction'),
        arrowprops=dict(arrowstyle='-|>', color='red', lw=3.0, shrinkA=0, shrinkB=0, mutation_scale=18),
        zorder=6
    )

    ax.set_xlabel("")  # on retire le label "Dates" si tu veux un rendu plus épuré
    ax.set_ylabel("")  # pas d'axe Y

    ax.set_title(title)

    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close(fig)

if __name__ == "__main__":
    # Exemple sans arguments : lecture directe d'un fichier par défaut
    input_csv = "example_tasks.csv"
    output_png = "planning_exemple.png"

    build_gantt(
        input_csv,
        output_png,
        show_labels_on_bars=True,
        draw_milestone_vlines=True,
        draw_row_background=True,
        row_bg_alpha=0.1,
        row_bg_height=0.6,
        bar_height=0.4,
        milestone_vline_alpha=0.3,
        # Texte dynamique
        max_font_size=12,
        min_font_size=6,
        label_padding_px=6,
        label_min_days_for_text=0.5,
    )

    print(f"Planning généré : {output_png}")

Planning généré : planning_exemple.png


## Quatrieme

In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.transforms as transforms
from matplotlib.lines import Line2D

def _data_width_in_pixels(ax, x_left, x_right):
    trans = ax.transData
    p0 = trans.transform((x_left, 0))
    p1 = trans.transform((x_right, 0))
    return abs(p1[0] - p0[0])

def _fit_text_inside_bar(ax, text_str, x_center, y_center, x_left, x_right,
                         max_font_size=12, min_font_size=6, padding_px=6, zorder=5):
    fig = ax.figure
    fig.canvas.draw()  # renderer prêt
    renderer = fig.canvas.get_renderer()

    available_px = max(_data_width_in_pixels(ax, x_left, x_right) - padding_px, 0)

    for fs in range(max_font_size, min_font_size - 1, -1):
        txt = ax.text(
            x_center, y_center, text_str,
            va="center", ha="center",
            fontsize=fs, clip_on=True, zorder=zorder
        )
        bbox = txt.get_window_extent(renderer=renderer)
        if bbox.width <= available_px:
            return True
        txt.remove()
    return False

def build_gantt(
    csv_path,
    png_path,
    *,
    # Titres sur les barres
    show_labels_on_bars=True,
    label_min_days_for_text=0.5,
    label_pad_days=0.0,
    max_font_size=12,
    min_font_size=6,
    label_padding_px=6,
    # Milestones
    draw_milestone_vlines=True,
    milestone_vline_linestyle=":",
    milestone_vline_alpha=0.3,
    milestone_marker="v",
    milestone_markersize=14,         # ++ taille
    milestone_colors=None,           # palette; None -> tab10
    milestone_on_axis_offset_af=0.02,# décalage vertical (axes fraction) pour que la pointe touche la flèche
    # Bandes de fond
    draw_row_background=True,
    row_bg_alpha=0.1,
    row_bg_height=0.6,
    # Barres
    bar_height=0.4,
    # Axe dates
    major_locator_weeks=1,
    date_fmt="%d %b %Y",
    fig_size=(12, 6),
    title="Planning",
    # Flèche d’axe X
    axis_arrow_color="red",
    axis_arrow_lw=6.0,               # ++ épaisseur
    axis_arrow_mutation_scale=28     # ++ taille de la pointe
):
    """
    CSV requis : id,title,start,end,group,milestone,depends_on
    - start/end: YYYY-MM-DD
    - milestone: 1 = jalon (start=end), 0 = tâche
    """

    df = pd.read_csv(csv_path)
    df["start"] = pd.to_datetime(df["start"])
    df["end"]   = pd.to_datetime(df["end"])
    df = df.sort_values("start").reset_index(drop=True)

    # Séparer tâches / jalons
    is_ms = (df.get("milestone", 0).astype(int) == 1) | (df["start"].dt.date == df["end"].dt.date)
    df_ms = df[is_ms].copy()
    df_tasks = df[~is_ms].copy()

    fig, ax = plt.subplots(figsize=fig_size)

    # Positions Y: réserver y=0 pour les milestones; les tâches commencent à y=1,2,3...
    y_positions = range(1, len(df_tasks) + 1)
    ax.set_yticks(list(y_positions))
    ax.set_yticklabels([])                 # pas de labels Y
    ax.tick_params(axis='y', left=False, labelleft=False)

    # Limites X globales
    x_min = mdates.date2num(df["start"].min())
    x_max = mdates.date2num(df["end"].max())

    # (Bandes de fond) pour les lignes de tâches uniquement
    if draw_row_background:
        for i in y_positions:
            y0 = i - row_bg_height / 2.0
            y1 = i + row_bg_height / 2.0
            ax.axhspan(y0, y1, xmin=0.0, xmax=1.0, alpha=row_bg_alpha, zorder=1)

    # Tracé des barres (tâches)
    for i, (idx, row) in enumerate(df_tasks.iterrows(), start=1):
        start_num = mdates.date2num(row["start"])
        end_num   = mdates.date2num(row["end"])
        duration  = end_num - start_num

        ax.barh(
            i,
            duration,
            left=start_num,
            height=bar_height,
            align='center',
            zorder=2
        )

        if show_labels_on_bars and duration >= label_min_days_for_text:
            label_x = start_num + label_pad_days + duration / 2.0
            _fit_text_inside_bar(
                ax,
                row["title"],
                label_x,
                i,
                start_num + label_pad_days,
                end_num,
                max_font_size=max_font_size,
                min_font_size=min_font_size,
                padding_px=label_padding_px,
                zorder=5
            )

    # Tracé des milestones :
    # - ligne verticale pointillée (optionnelle)
    # - triangle inversé posé sur la flèche d’axe X (y en "axes fraction")
    # Couleurs distinctes + légende
    if milestone_colors is None:
        # palette discrète
        from itertools import cycle
        palette = plt.get_cmap("tab10").colors
        color_cycle = cycle(palette)
        ms_colors = [next(color_cycle) for _ in range(len(df_ms))]
    else:
        ms_colors = milestone_colors

    legend_handles = []
    blended = transforms.blended_transform_factory(ax.transData, ax.transAxes)

    for (row, color) in zip(df_ms.itertuples(index=False), ms_colors):
        x_ms = mdates.date2num(row.start)

        # ligne verticale
        if draw_milestone_vlines:
            ax.axvline(x=x_ms, linestyle=milestone_vline_linestyle,
                       alpha=milestone_vline_alpha, zorder=3)

        # triangle inversé sur la flèche: y en axes-fraction (0 = bas)
        # on met un léger offset positif pour que la POINTE touche la flèche rouge.
        ax.plot(
            [x_ms], [milestone_on_axis_offset_af],
            marker=milestone_marker,
            markersize=milestone_markersize,
            markerfacecolor=color,
            markeredgecolor=color,
            transform=blended,
            zorder=7
        )

        # pour la légende
        legend_handles.append(
            Line2D([0], [0],
                   marker=milestone_marker, linestyle='None',
                   markersize=milestone_markersize,
                   markerfacecolor=color, markeredgecolor=color,
                   label=row.title)
        )

    if legend_handles:
        ax.legend(handles=legend_handles, loc="upper left", frameon=False)

    # Axe X (dates)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(-0.5, len(df_tasks) + 0.8)  # un peu d'air au-dessus
    ax.xaxis_date()
    if major_locator_weeks is not None:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=major_locator_weeks))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    # Supprimer le cadre (spines)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remplacer l’axe X par une flèche rouge épaisse au BAS du graphe
    ax.annotate(
        '',
        xy=(x_max, 0.0), xycoords=('data', 'axes fraction'),
        xytext=(x_min, 0.0), textcoords=('data', 'axes fraction'),
        arrowprops=dict(arrowstyle='-|>',
                        color=axis_arrow_color,
                        lw=axis_arrow_lw,
                        shrinkA=0, shrinkB=0,
                        mutation_scale=axis_arrow_mutation_scale),
        zorder=6
    )

    # Pas d’axe Y explicite
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title(title)

    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close(fig)

if __name__ == "__main__":
    # Exemple sans arguments : lecture directe d'un fichier par défaut
    input_csv = "example_tasks.csv"
    output_png = "planning_exemple.png"

    build_gantt(
        input_csv,
        output_png,
        show_labels_on_bars=True,
        draw_milestone_vlines=True,
        draw_row_background=True,
        row_bg_alpha=0.1,
        row_bg_height=0.6,
        bar_height=0.4,
        # Texte
        max_font_size=12,
        min_font_size=6,
        label_padding_px=6,
        label_min_days_for_text=0.5,
        # Milestones
        milestone_markersize=16,
        milestone_on_axis_offset_af=0.02,
        # Flèche d’axe X
        axis_arrow_color="red",
        axis_arrow_lw=6.0,
        axis_arrow_mutation_scale=28,
        title="Planning"
    )

    print(f"Planning généré : {output_png}")

Planning généré : planning_exemple.png


## Cinquièeme 

In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.transforms as transforms
from matplotlib.lines import Line2D

def _data_width_in_pixels(ax, x_left, x_right):
    trans = ax.transData
    p0 = trans.transform((x_left, 0))
    p1 = trans.transform((x_right, 0))
    return abs(p1[0] - p0[0])

def _fit_text_inside_bar(ax, text_str, x_center, y_center, x_left, x_right,
                         max_font_size=12, min_font_size=6, padding_px=6, zorder=5):
    fig = ax.figure
    fig.canvas.draw()  # renderer prêt
    renderer = fig.canvas.get_renderer()

    available_px = max(_data_width_in_pixels(ax, x_left, x_right) - padding_px, 0)

    for fs in range(max_font_size, min_font_size - 1, -1):
        txt = ax.text(
            x_center, y_center, text_str,
            va="center", ha="center",
            fontsize=fs, clip_on=True, zorder=zorder
        )
        bbox = txt.get_window_extent(renderer=renderer)
        if bbox.width <= available_px:
            return True
        txt.remove()
    return False

def build_gantt(
    csv_path,
    png_path,
    *,
    # Titres sur les barres
    show_labels_on_bars=True,
    label_min_days_for_text=0.5,
    label_pad_days=0.0,
    max_font_size=12,
    min_font_size=6,
    label_padding_px=6,
    # Milestones
    draw_milestone_vlines=True,
    milestone_vline_linestyle=":",
    milestone_vline_alpha=0.3,
    milestone_marker="v",
    milestone_markersize=16,
    milestone_colors=None,            # None -> palette tab10
    milestone_on_axis_offset_af=0.02, # position verticale des triangles (axes fraction)
    # Bandes de fond
    draw_row_background=True,
    row_bg_alpha=0.1,
    row_bg_height=0.6,
    # Barres
    bar_height=0.4,
    # Axe dates
    major_locator_weeks=1,
    date_fmt="%d %b %Y",
    fig_size=(12, 6),
    title="Planning",
    # Flèche d’axe X
    axis_arrow_color="red",
    axis_arrow_lw=6.0,
    axis_arrow_mutation_scale=28,
    # Marges horizontales (avant/après) en proportion de la largeur totale
    x_margin_ratio=0.05
):
    """
    CSV requis : id,title,start,end,group,milestone,depends_on
    - start/end: YYYY-MM-DD
    - milestone: 1 = jalon (start=end), 0 = tâche
    """

    df = pd.read_csv(csv_path)
    df["start"] = pd.to_datetime(df["start"])
    df["end"]   = pd.to_datetime(df["end"])
    df = df.sort_values("start").reset_index(drop=True)

    # Séparer tâches / jalons
    is_ms = (df.get("milestone", 0).astype(int) == 1) | (df["start"].dt.date == df["end"].dt.date)
    df_ms = df[is_ms].copy()
    df_tasks = df[~is_ms].copy()

    fig, ax = plt.subplots(figsize=fig_size)

    # Positions Y (y=0 réservé aux milestones)
    y_positions = range(1, len(df_tasks) + 1)
    ax.set_yticks(list(y_positions))
    ax.set_yticklabels([])                 # pas de labels Y
    ax.tick_params(axis='y', left=False, labelleft=False)

    # Limites X globales (incluant tâches + jalons)
    x_min_raw = mdates.date2num(df["start"].min())
    x_max_raw = mdates.date2num(df["end"].max())

    total_range = x_max_raw - x_min_raw
    if total_range == 0:
        # cas degénéré : ajouter un jour pour éviter plage nulle
        total_range = 1.0
        x_min_raw -= 0.5
        x_max_raw += 0.5

    # Ajouter marges aux extrémités
    margin = total_range * x_margin_ratio
    x_min = x_min_raw - margin
    x_max = x_max_raw + margin

    # (Bandes de fond) pour les lignes de tâches uniquement
    if draw_row_background:
        for i in y_positions:
            y0 = i - row_bg_height / 2.0
            y1 = i + row_bg_height / 2.0
            ax.axhspan(y0, y1, xmin=0.0, xmax=1.0, alpha=row_bg_alpha, zorder=1)

    # Tracé des barres (tâches)
    for i, (idx, row) in enumerate(df_tasks.iterrows(), start=1):
        start_num = mdates.date2num(row["start"])
        end_num   = mdates.date2num(row["end"])
        duration  = end_num - start_num

        ax.barh(
            i,
            duration,
            left=start_num,
            height=bar_height,
            align='center',
            zorder=2
        )

        if show_labels_on_bars and duration >= label_min_days_for_text:
            label_x = start_num + label_pad_days + duration / 2.0
            _fit_text_inside_bar(
                ax,
                row["title"],
                label_x,
                i,
                start_num + label_pad_days,
                end_num,
                max_font_size=max_font_size,
                min_font_size=min_font_size,
                padding_px=label_padding_px,
                zorder=5
            )

    # Tracé des milestones à y=0
    if milestone_colors is None:
        from itertools import cycle
        palette = plt.get_cmap("tab10").colors
        color_cycle = cycle(palette)
        ms_colors = [next(color_cycle) for _ in range(len(df_ms))]
    else:
        ms_colors = milestone_colors

    legend_handles = []
    blended = transforms.blended_transform_factory(ax.transData, ax.transAxes)

    for (row, color) in zip(df_ms.itertuples(index=False), ms_colors):
        x_ms = mdates.date2num(row.start)

        # Ligne verticale pointillée (optionnelle)
        if draw_milestone_vlines:
            ax.axvline(x=x_ms, linestyle=milestone_vline_linestyle,
                       alpha=milestone_vline_alpha, zorder=3)

        # Triangle inversé posé sur la flèche (bas du graphe), avec léger offset
        ax.plot(
            [x_ms], [milestone_on_axis_offset_af],
            marker=milestone_marker,
            markersize=milestone_markersize,
            markerfacecolor=color,
            markeredgecolor=color,
            transform=blended,
            zorder=7
        )

        # Légende
        legend_handles.append(
            Line2D([0], [0],
                   marker=milestone_marker, linestyle='None',
                   markersize=milestone_markersize,
                   markerfacecolor=color, markeredgecolor=color,
                   label=row.title)
        )

    if legend_handles:
        ax.legend(handles=legend_handles, loc="upper left", frameon=False)

    # Axe X (dates)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(-0.5, len(df_tasks) + 0.8)
    ax.xaxis_date()
    if major_locator_weeks is not None:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=major_locator_weeks))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    # Supprimer le cadre (spines)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Flèche rouge de l’axe X (utilise x_min/x_max margés)
    ax.annotate(
        '',
        xy=(x_max, 0.0), xycoords=('data', 'axes fraction'),
        xytext=(x_min, 0.0), textcoords=('data', 'axes fraction'),
        arrowprops=dict(arrowstyle='-|>',
                        color=axis_arrow_color,
                        lw=axis_arrow_lw,
                        shrinkA=0, shrinkB=0,
                        mutation_scale=axis_arrow_mutation_scale),
        zorder=6
    )

    # Pas d’axe Y explicite
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title(title)

    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close(fig)

if __name__ == "__main__":
    # Exemple sans arguments : lecture directe d'un fichier par défaut
    input_csv = "example_tasks.csv"
    output_png = "planning_exemple.png"

    build_gantt(
        input_csv,
        output_png,
        show_labels_on_bars=True,
        draw_milestone_vlines=True,
        draw_row_background=True,
        row_bg_alpha=0.1,
        row_bg_height=0.6,
        bar_height=0.4,
        # Texte
        max_font_size=12,
        min_font_size=6,
        label_padding_px=6,
        label_min_days_for_text=0.5,
        # Milestones
        milestone_markersize=16,
        milestone_on_axis_offset_af=0.02,
        # Flèche d’axe X
        axis_arrow_color="red",
        axis_arrow_lw=8.0,
        axis_arrow_mutation_scale=30,
        # Marge horizontale
        x_margin_ratio=0.05,
        title="Planning"
    )

    print(f"Planning généré : {output_png}")

Planning généré : planning_exemple.png


## Sixième

In [17]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.transforms as transforms
from matplotlib.lines import Line2D

def _data_width_in_pixels(ax, x_left, x_right):
    trans = ax.transData
    p0 = trans.transform((x_left, 0))
    p1 = trans.transform((x_right, 0))
    return abs(p1[0] - p0[0])

def _fit_text_inside_bar(ax, text_str, x_center, y_center, x_left, x_right,
                         max_font_size=12, min_font_size=6, padding_px=6, zorder=5):
    """Ajuste dynamiquement la police pour que text_str tienne en largeur dans [x_left, x_right]."""
    fig = ax.figure
    fig.canvas.draw()
    renderer = fig.canvas.get_renderer()
    available_px = max(_data_width_in_pixels(ax, x_left, x_right) - padding_px, 0)

    for fs in range(max_font_size, min_font_size - 1, -1):
        txt = ax.text(
            x_center, y_center, text_str,
            va="center", ha="center",
            fontsize=fs, clip_on=True, zorder=zorder
        )
        bbox = txt.get_window_extent(renderer=renderer)
        if bbox.width <= available_px:
            return fs, txt  # retourne la taille retenue et l'objet texte (laissé sur l'axe)
        txt.remove()
    return None, None  # trop long, on n'affiche pas

def _text_height_in_data(ax, px):
    """Convertit un offset vertical en pixels vers des unités Y (data)."""
    # transforme (0,0) et (0,1) pour connaitre pixels par unité Y
    fig = ax.figure
    fig.canvas.draw()
    p0 = ax.transData.transform((0, 0))
    p1 = ax.transData.transform((0, 1))
    py_per_data = abs(p1[1] - p0[1])
    if py_per_data == 0:
        return 0.0
    return px / py_per_data

def days_between(d0, d1, inclusive=True):
    """Calcule la durée en jours. inclusive=True -> +1 jour si même date."""
    delta = (d1 - d0).days
    return delta + 1 if inclusive else delta

def build_gantt(
    csv_path,
    png_path,
    *,
    # Titres sur les barres
    show_labels_on_bars=True,
    label_min_days_for_text=0.5,
    label_pad_days=0.0,
    max_font_size=12,
    min_font_size=6,
    label_padding_px=6,
    # Affichage de la durée sous le titre
    show_duration_under_title=True,
    duration_inclusive=True,             # compte les jours de début et de fin
    duration_label_fmt="{d} j",          # format du texte durée
    duration_font_scale=0.85,            # taille du texte durée vs titre
    duration_vertical_gap_px=10,         # écart vertical (pixels) entre titre et durée
    # Milestones
    draw_milestone_vlines=True,
    milestone_vline_linestyle=":",
    milestone_vline_alpha=0.3,
    milestone_marker="v",
    milestone_markersize=16,
    milestone_colors=None,               # None -> tab10
    milestone_on_axis_offset_af=0.02,    # position verticale des triangles (axes fraction)
    # Bandes de fond
    draw_row_background=True,
    row_bg_alpha=0.1,
    row_bg_height=0.6,
    # Barres
    bar_height=0.4,
    # Dates début/fin visibles sur les barres
    show_start_end_date_labels=True,
    start_end_date_fmt="%d/%m",
    date_label_offset_days=0.2,          # décalage horizontal des étiquettes
    draw_task_endcaps=True,              # petits traits verticaux aux extrémités
    endcap_length_data=0.35,             # demi-hauteur des end-caps (unités Y)
    # Axe dates
    major_locator_weeks=1,
    date_fmt="%d %b %Y",
    fig_size=(12, 6),
    title="Planning",
    # Flèche d’axe X
    axis_arrow_color="red",
    axis_arrow_lw=8.0,
    axis_arrow_mutation_scale=30,
    # Marges horizontales
    x_margin_ratio=0.05
):
    """
    CSV requis : id,title,start,end,group,milestone,depends_on
    - start/end: YYYY-MM-DD
    - milestone: 1 = jalon (start=end), 0 = tâche
    """

    df = pd.read_csv(csv_path)
    df["start"] = pd.to_datetime(df["start"])
    df["end"]   = pd.to_datetime(df["end"])
    df = df.sort_values("start").reset_index(drop=True)

    # Séparer tâches / jalons
    is_ms = (df.get("milestone", 0).astype(int) == 1) | (df["start"].dt.date == df["end"].dt.date)
    df_ms = df[is_ms].copy()
    df_tasks = df[~is_ms].copy()

    fig, ax = plt.subplots(figsize=fig_size)

    # Positions Y (y=0 réservé aux milestones)
    y_positions = range(1, len(df_tasks) + 1)
    ax.set_yticks(list(y_positions))
    ax.set_yticklabels([])
    ax.tick_params(axis='y', left=False, labelleft=False)

    # Limites X globales (incluant tâches + jalons)
    x_min_raw = mdates.date2num(df["start"].min())
    x_max_raw = mdates.date2num(df["end"].max())
    total_range = x_max_raw - x_min_raw
    if total_range == 0:
        total_range = 1.0
        x_min_raw -= 0.5
        x_max_raw += 0.5

    margin = total_range * x_margin_ratio
    x_min = x_min_raw - margin
    x_max = x_max_raw + margin

    # Bandes de fond
    if draw_row_background:
        for i in y_positions:
            y0 = i - row_bg_height / 2.0
            y1 = i + row_bg_height / 2.0
            ax.axhspan(y0, y1, xmin=0.0, xmax=1.0, alpha=row_bg_alpha, zorder=1)

    # Tracé des barres (tâches) + titres + durée + dates début/fin
    for i, (idx, row) in enumerate(df_tasks.iterrows(), start=1):
        start_num = mdates.date2num(row["start"])
        end_num   = mdates.date2num(row["end"])
        duration_days = days_between(row["start"].date(), row["end"].date(), inclusive=duration_inclusive)
        duration = end_num - start_num

        # Barre
        ax.barh(
            i,
            duration,
            left=start_num,
            height=bar_height,
            align='center',
            zorder=2
        )

        # Titre (avec auto-fit), puis durée en plus petit juste en dessous
        if show_labels_on_bars and duration >= label_min_days_for_text:
            label_x = start_num + label_pad_days + duration / 2.0
            # 1) Titre
            fs_used, title_text = _fit_text_inside_bar(
                ax,
                row["title"],
                label_x,
                i,
                start_num + label_pad_days,
                end_num,
                max_font_size=max_font_size,
                min_font_size=min_font_size,
                padding_px=label_padding_px,
                zorder=5
            )
            # 2) Durée (si demandée)
            if fs_used and show_duration_under_title:
                # place la durée quelques pixels sous le titre
                dy = _text_height_in_data(ax, duration_vertical_gap_px)
                dur_fs = max(int(fs_used * duration_font_scale), min_font_size)
                dur_str = duration_label_fmt.format(d=duration_days)
                dur_txt = ax.text(
                    label_x, i - dy,
                    dur_str,
                    va="center", ha="center",
                    fontsize=dur_fs, clip_on=True, zorder=5
                )

        # Dates début/fin visibles + end-caps
        if show_start_end_date_labels:
            date_left  = mdates.num2date(start_num).strftime(start_end_date_fmt)
            date_right = mdates.num2date(end_num).strftime(start_end_date_fmt)
            # Etiquettes de part et d'autre de la barre
            ax.text(
                start_num - date_label_offset_days, i,
                date_left,
                va="center", ha="right",
                fontsize=9, zorder=5, clip_on=False
            )
            ax.text(
                end_num + date_label_offset_days, i,
                date_right,
                va="center", ha="left",
                fontsize=9, zorder=5, clip_on=False
            )
        if draw_task_endcaps:
            # petits traits verticaux aux extrémités
            ax.vlines([start_num, end_num],
                      ymin=i - endcap_length_data, ymax=i + endcap_length_data,
                      zorder=5)

    # Tracé des milestones à y=0
    if milestone_colors is None:
        from itertools import cycle
        palette = plt.get_cmap("tab10").colors
        color_cycle = cycle(palette)
        ms_colors = [next(color_cycle) for _ in range(len(df_ms))]
    else:
        ms_colors = milestone_colors

    legend_handles = []
    blended = transforms.blended_transform_factory(ax.transData, ax.transAxes)

    for (row, color) in zip(df_ms.itertuples(index=False), ms_colors):
        x_ms = mdates.date2num(row.start)

        if draw_milestone_vlines:
            ax.axvline(x=x_ms, linestyle=milestone_vline_linestyle,
                       alpha=milestone_vline_alpha, zorder=3)

        ax.plot(
            [x_ms], [milestone_on_axis_offset_af],
            marker="v",
            markersize=milestone_markersize,
            markerfacecolor=color,
            markeredgecolor=color,
            transform=blended,
            zorder=7
        )

        legend_handles.append(
            Line2D([0], [0],
                   marker="v", linestyle='None',
                   markersize=milestone_markersize,
                   markerfacecolor=color, markeredgecolor=color,
                   label=row.title)
        )

    if legend_handles:
        ax.legend(handles=legend_handles, loc="upper left", frameon=False)

    # Axe X (dates)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(-0.5, len(df_tasks) + 0.8)
    ax.xaxis_date()
    if major_locator_weeks is not None:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=major_locator_weeks))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    # Supprime le cadre
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Flèche rouge épaisse en bas (utilise x_min/x_max margés)
    ax.annotate(
        '',
        xy=(x_max, 0.0), xycoords=('data', 'axes fraction'),
        xytext=(x_min, 0.0), textcoords=('data', 'axes fraction'),
        arrowprops=dict(arrowstyle='-|>',
                        color=axis_arrow_color,
                        lw=axis_arrow_lw,
                        shrinkA=0, shrinkB=0,
                        mutation_scale=axis_arrow_mutation_scale),
        zorder=6
    )

    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title(title)

    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close(fig)

if __name__ == "__main__":
    # Exemple sans arguments : lecture directe d'un fichier par défaut
    input_csv = "example_tasks.csv"
    output_png = "planning_exemple.png"

    build_gantt(
        input_csv,
        output_png,
        show_labels_on_bars=True,
        draw_milestone_vlines=True,
        draw_row_background=True,
        row_bg_alpha=0.1,
        row_bg_height=0.6,
        bar_height=0.4,
        # Titre + durée
        max_font_size=12,
        min_font_size=6,
        label_padding_px=6,
        label_min_days_for_text=0.5,
        show_duration_under_title=True,
        duration_inclusive=True,
        duration_label_fmt="{d} j",
        duration_font_scale=0.85,
        duration_vertical_gap_px=10,
        # Dates début/fin + end-caps
        show_start_end_date_labels=True,
        start_end_date_fmt="%d/%m",
        date_label_offset_days=0.2,
        draw_task_endcaps=True,
        endcap_length_data=0.35,
        # Milestones
        milestone_markersize=16,
        milestone_on_axis_offset_af=0.02,
        # Flèche d’axe X
        axis_arrow_color="red",
        axis_arrow_lw=8.0,
        axis_arrow_mutation_scale=30,
        # Marge horizontale
        x_margin_ratio=0.05,
        title="Planning"
    )

    print(f"Planning généré : {output_png}")

Planning généré : planning_exemple.png


## Septièeme

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.transforms as transforms
from matplotlib.lines import Line2D

def _data_width_in_pixels(ax, x_left, x_right):
    trans = ax.transData
    p0 = trans.transform((x_left, 0))
    p1 = trans.transform((x_right, 0))
    return abs(p1[0] - p0[0])

def _fit_text_in_span(ax, text_str, x_center, y, x_left, x_right,
                      max_font_size=12, min_font_size=6, padding_px=6, zorder=5,
                      ha="center", va="center", clip_on=True):
    """
    Place un texte à (x_center, y) et réduit la police si nécessaire pour que le texte
    tienne entre x_left et x_right. Retourne (fs_utilisée, handle_texte) ou (None, None).
    """
    fig = ax.figure
    fig.canvas.draw()
    renderer = fig.canvas.get_renderer()
    available_px = max(_data_width_in_pixels(ax, x_left, x_right) - padding_px, 0)

    for fs in range(max_font_size, min_font_size - 1, -1):
        txt = ax.text(
            x_center, y, text_str,
            va=va, ha=ha,
            fontsize=fs, clip_on=clip_on, zorder=zorder
        )
        bbox = txt.get_window_extent(renderer=renderer)
        if bbox.width <= available_px:
            return fs, txt
        txt.remove()
    return None, None

def _px_to_data_y(ax, px):
    """Convertit un offset vertical en pixels vers des unités Y (data)."""
    fig = ax.figure
    fig.canvas.draw()
    p0 = ax.transData.transform((0, 0))
    p1 = ax.transData.transform((0, 1))
    py_per_data = abs(p1[1] - p0[1])
    return 0.0 if py_per_data == 0 else px / py_per_data

def days_between(d0, d1, inclusive=True):
    delta = (d1 - d0).days
    return delta + 1 if inclusive else delta

def build_gantt(
    csv_path,
    png_path,
    *,
    # ----- Titres (au-dessus des barres) -----
    show_titles_above_bars=True,
    title_above_max_font_size=10,
    title_above_min_font_size=6,
    title_above_padding_px=6,
    # position verticale du titre au-dessus de la barre :
    #   y = i + (row_bg_height/2) + title_above_gap_px (converti en unités data)
    title_above_gap_px=6,
    # ----- Durée dans la barre -----
    show_duration_inside_bar=True,
    duration_inclusive=True,
    duration_label_fmt="{d} j",
    duration_font_size=8,
    # ----- Ancien titre dans la barre (désactivé par défaut) -----
    show_labels_on_bars=False,  # <<< on désactive l'ancien comportement
    label_min_days_for_text=0.5,
    label_pad_days=0.0,
    max_font_size=12,
    min_font_size=6,
    label_padding_px=6,
    # ----- Milestones -----
    draw_milestone_vlines=True,
    milestone_vline_linestyle=":",
    milestone_vline_alpha=0.3,
    milestone_marker="v",
    milestone_markersize=16,
    milestone_colors=None,            # None -> tab10
    milestone_on_axis_offset_af=0.02,
    # ----- Bandes de fond -----
    draw_row_background=True,
    row_bg_alpha=0.1,
    row_bg_height=0.6,
    # ----- Barres -----
    bar_height=0.4,
    # ----- Dates début/fin & end-caps -----
    show_start_end_date_labels=True,
    start_end_date_fmt="%d/%m",
    date_label_offset_days=0.2,
    draw_task_endcaps=True,
    endcap_length_data=0.35,
    # ----- Axe dates -----
    major_locator_weeks=1,
    date_fmt="%d %b %Y",
    fig_size=(12, 6),
    title="Planning",
    # ----- Flèche axe X -----
    axis_arrow_color="red",
    axis_arrow_lw=8.0,
    axis_arrow_mutation_scale=30,
    # ----- Marges horizontales -----
    x_margin_ratio=0.05
):
    """
    CSV requis : id,title,start,end,group,milestone,depends_on
    - start/end: YYYY-MM-DD
    - milestone: 1 = jalon (start=end), 0 = tâche
    """

    df = pd.read_csv(csv_path)
    df["start"] = pd.to_datetime(df["start"])
    df["end"]   = pd.to_datetime(df["end"])
    df = df.sort_values("start").reset_index(drop=True)

    # Séparer tâches / jalons
    is_ms = (df.get("milestone", 0).astype(int) == 1) | (df["start"].dt.date == df["end"].dt.date)
    df_ms = df[is_ms].copy()
    df_tasks = df[~is_ms].copy()

    fig, ax = plt.subplots(figsize=fig_size)

    # Y (y=0 pour milestones), cacher l'axe Y
    y_positions = range(1, len(df_tasks) + 1)
    ax.set_yticks(list(y_positions))
    ax.set_yticklabels([])
    ax.tick_params(axis='y', left=False, labelleft=False)

    # Limites X + marges
    x_min_raw = mdates.date2num(df["start"].min())
    x_max_raw = mdates.date2num(df["end"].max())
    total_range = x_max_raw - x_min_raw
    if total_range == 0:
        total_range = 1.0
        x_min_raw -= 0.5
        x_max_raw += 0.5
    margin = total_range * x_margin_ratio
    x_min = x_min_raw - margin
    x_max = x_max_raw + margin

    # Bandes de fond
    if draw_row_background:
        for i in y_positions:
            y0 = i - row_bg_height / 2.0
            y1 = i + row_bg_height / 2.0
            ax.axhspan(y0, y1, xmin=0.0, xmax=1.0, alpha=row_bg_alpha, zorder=1)

    # Tracé des tâches
    for i, (idx, row) in enumerate(df_tasks.iterrows(), start=1):
        start_num = mdates.date2num(row["start"])
        end_num   = mdates.date2num(row["end"])
        duration_days = days_between(row["start"].date(), row["end"].date(), inclusive=duration_inclusive)
        duration_num  = end_num - start_num

        # Barre
        ax.barh(
            i,
            duration_num,
            left=start_num,
            height=bar_height,
            align='center',
            zorder=2
        )

        # (A) Titre AU-DESSUS de la barre (dans la zone blanche), centré sur la durée
        if show_titles_above_bars:
            # position verticale au-dessus de la bande de fond (côté supérieur)
            y_title = i + (row_bg_height / 2.0) + _px_to_data_y(ax, title_above_gap_px)
            _fit_text_in_span(
                ax,
                row["title"],
                x_center=start_num + duration_num / 2.0,
                y=y_title,
                x_left=start_num,
                x_right=end_num,
                max_font_size=title_above_max_font_size,
                min_font_size=title_above_min_font_size,
                padding_px=title_above_padding_px,
                zorder=6,
                ha="center",
                va="center",
                clip_on=False  # on est dans la zone "au-dessus", pas besoin de clip
            )

        # (B) Durée DANS la barre (petite police)
        if show_duration_inside_bar and duration_num > 0:
            ax.text(
                start_num + duration_num / 2.0,
                i,
                duration_label_fmt.format(d=duration_days),
                va="center", ha="center",
                fontsize=duration_font_size, clip_on=True, zorder=5
            )

        # (C) Dates début/fin + end-caps
        if show_start_end_date_labels:
            date_left  = mdates.num2date(start_num).strftime(start_end_date_fmt)
            date_right = mdates.num2date(end_num).strftime(start_end_date_fmt)
            ax.text(
                start_num - date_label_offset_days, i,
                date_left,
                va="center", ha="right",
                fontsize=9, zorder=5, clip_on=False
            )
            ax.text(
                end_num + date_label_offset_days, i,
                date_right,
                va="center", ha="left",
                fontsize=9, zorder=5, clip_on=False
            )
        if draw_task_endcaps:
            ax.vlines([start_num, end_num],
                      ymin=i - endcap_length_data, ymax=i + endcap_length_data,
                      zorder=5)

    # Tracé des milestones à y=0
    if milestone_colors is None:
        from itertools import cycle
        palette = plt.get_cmap("tab10").colors
        color_cycle = cycle(palette)
        ms_colors = [next(color_cycle) for _ in range(len(df_ms))]
    else:
        ms_colors = milestone_colors

    legend_handles = []
    blended = transforms.blended_transform_factory(ax.transData, ax.transAxes)

    for (row, color) in zip(df_ms.itertuples(index=False), ms_colors):
        x_ms = mdates.date2num(row.start)

        if draw_milestone_vlines:
            ax.axvline(x=x_ms, linestyle=milestone_vline_linestyle,
                       alpha=milestone_vline_alpha, zorder=3)

        ax.plot(
            [x_ms], [milestone_on_axis_offset_af],
            marker=milestone_marker,
            markersize=milestone_markersize,
            markerfacecolor=color,
            markeredgecolor=color,
            transform=blended,
            zorder=7
        )

        legend_handles.append(
            Line2D([0], [0],
                   marker=milestone_marker, linestyle='None',
                   markersize=milestone_markersize,
                   markerfacecolor=color, markeredgecolor=color,
                   label=row.title)
        )

    if legend_handles:
        ax.legend(handles=legend_handles, loc="upper left", frameon=False)

    # Axe X (dates)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(-0.5, len(df_tasks) + 0.8)
    ax.xaxis_date()
    if major_locator_weeks is not None:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=major_locator_weeks))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    # Supprimer le cadre
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Flèche rouge épaisse (bas)
    ax.annotate(
        '',
        xy=(x_max, 0.0), xycoords=('data', 'axes fraction'),
        xytext=(x_min, 0.0), textcoords=('data', 'axes fraction'),
        arrowprops=dict(arrowstyle='-|>',
                        color=axis_arrow_color,
                        lw=axis_arrow_lw,
                        shrinkA=0, shrinkB=0,
                        mutation_scale=axis_arrow_mutation_scale),
        zorder=6
    )

    # Pas d’axe Y explicite
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title(title)

    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close(fig)

if __name__ == "__main__":
    # Exemple sans arguments : lecture directe d'un fichier par défaut
    input_csv = "example_tasks.csv"
    output_png = "planning_exemple.png"

    build_gantt(
        input_csv,
        output_png,
        # Titres au-dessus
        show_titles_above_bars=True,
        title_above_max_font_size=10,
        title_above_min_font_size=6,
        title_above_padding_px=6,
        title_above_gap_px=6,
        # Durée dans la barre
        show_duration_inside_bar=True,
        duration_inclusive=True,
        duration_label_fmt="{d} j",
        duration_font_size=8,
        # Autres options déjà paramétrées
        draw_milestone_vlines=True,
        draw_row_background=True,
        row_bg_alpha=0.1,
        row_bg_height=0.6,
        bar_height=0.4,
        # Dates/Endcaps
        show_start_end_date_labels=True,
        start_end_date_fmt="%d/%m",
        date_label_offset_days=0.2,
        draw_task_endcaps=True,
        endcap_length_data=0.35,
        # Milestones
        milestone_markersize=16,
        milestone_on_axis_offset_af=0.02,
        # Axe X
        axis_arrow_color="red",
        axis_arrow_lw=8.0,
        axis_arrow_mutation_scale=30,
        # Marges horizontales
        x_margin_ratio=0.05,
        title="Planning"
    )

    print(f"Planning généré : {output_png}")

Planning généré : planning_exemple.png
